In [2]:
import requests
import pandas as pd
from datetime import datetime
import sqlite3

📊 Variables Macro para Modelo de Credit Scoring

A continuación se presentan los indicadores macroeconómicos recomendados para incluir en el modelo. Cada variable indica qué mide, frecuencia, justificación y fuente/API.

1️⃣ PIB Real de España

Qué mide: Nivel de actividad económica del país.

Frecuencia: Trimestral

Justificación: Más crecimiento económico → más empleo → menor riesgo de impago.

Fuente/API: INE – API de series temporales

2️⃣ Tasa de desempleo

Qué mide: Porcentaje de población desempleada.

Frecuencia: Trimestral

Justificación: Más desempleo → menor capacidad de pago → mayor riesgo de impago.

Fuente/API: INE / Eurostat – Eurostat JSON API

3️⃣ Índice de Precios al Consumidor (IPC)

Qué mide: Nivel general de inflación.

Frecuencia: Mensual

Justificación: Inflación alta reduce poder adquisitivo → puede afectar la capacidad de pago.

Fuente/API: INE – API de series temporales

4️⃣ Euribor 12M

Qué mide: Tipo de interés de referencia en préstamos a un año.

Frecuencia: Diario / Mensual

Justificación: Tipos más altos → préstamos más caros → mayor riesgo de impago.

Fuente/API: Banco Central Europeo – ECB SDW API

5️⃣ Tipo de cambio EUR/USD

Qué mide: Valor del euro frente al dólar.

Frecuencia: Diario

Justificación: Afecta importaciones/exportaciones y economía general → capacidad de pago.

Fuente/API: ECB SDW API / Fixer.io API

6️⃣ Precio del petróleo Brent

Qué mide: Coste de energía y transporte.

Frecuencia: Diario

Justificación: Petróleo caro → costos más altos → presión económica → riesgo de impago.

Fuente/API: EIA – Open Data API

7️⃣ IBEX 35

Qué mide: Indicador de mercado bursátil español.

Frecuencia: Diario

Justificación: Señal de confianza económica; caídas pueden reflejar incertidumbre → mayor riesgo.

Fuente/API: Yahoo Finance – Yahoo Finance API

8️⃣ Spread de bonos España 10y vs Bund

Qué mide: Diferencial de riesgo país España vs Alemania.

Frecuencia: Diario / Mensual

Justificación: Spread alto → incertidumbre económica → afecta capacidad de pago.

Fuente/API: ECB SDW API

9️⃣ Índice de producción industrial

Qué mide: Actividad del sector manufacturero.

Frecuencia: Mensual

Justificación: Mayor producción → economía más fuerte → menor riesgo de impago.

Fuente/API: Eurostat – API de Eurostat

🔟 Crecimiento sectorial (construcción y servicios)

Qué mide: Nivel de actividad de sectores clave.

Frecuencia: Trimestral

Justificación: Dinamismo sectorial → empleo y capacidad de pago más altos.

Fuente/API: Eurostat – API de Eurostat